In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy.stats
import geopandas as gpd
%matplotlib inline
from IPython.display import Markdown
from functools import reduce
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

### Theming

In [35]:
if 1:
    style = {
        # 'background-color': '#1b1b1b', # rgb(27, 27, 27)
        'color': 'white', # font
    }
    pio.templates.default = 'plotly_dark'
else:
    style = None
    pio.templates.default = 'plotly'

### Loading
Table 40 by kraje/regions  
Age of 90 will mean age of 90 or more

In [36]:
import sys
sys.path.append('../')

from preprocessing import rename_columns, replace_with_nan
table_40 = pd.read_csv('../data/RV_O_040_R_KR_SK.CSV', sep=';')
table_40['Vek'] = pd.to_numeric(table_40['Vek'].replace('90 a viac rokov', '90'))

## Grouping education

Grouping to
- unspecified
- without education
- primary education
- secondary education
- higher education

In [62]:
education_levels = set(table_40['Vzdelanie'].unique())
display(education_levels)

grouped_edu = 'Grouped edu'

primary = [x for x in education_levels if "základné" in x]
education_levels -= set(primary) # for quering purposes
secondary = [x for x in education_levels if "stredné" in x]
education_levels -= set(secondary)
higher = [x for x in education_levels if 'vysokoškolské' in x or "vyššie" in x]
education_levels -= set(higher)
without = [x for x in education_levels if 'bez' in x]
education_levels -= set(without)
unspecified = ['nezistené']
education_levels -= set(unspecified)
display(education_levels)

for label, level in zip(['primary', 'secondary', 'higher', 'without', 'unspecified'],
                        [primary, secondary, higher, without, unspecified]):
    level = table_40['Vzdelanie'].isin(level)
    table_40.loc[level, grouped_edu] = label

# table_40.sample(30)


{'bez ukončeného vzdelania – osoby vo veku 0-14 rokov',
 'bez školského vzdelania – osoby vo veku 15 rokov a viac',
 'nezistené',
 'stredné odborné (učňovské) vzdelanie bez maturity (bližšie neuvedené)',
 'stredné odborné (učňovské) vzdelanie bez maturity a bez výučného listu (zaškolenie, zaučenie)',
 'stredné odborné (učňovské) vzdelanie bez maturity s vysvedčením o záverečnej skúške',
 'stredné odborné (učňovské) vzdelanie bez maturity s výučným listom',
 'vysokoškolské vzdelanie (bližšie neuvedené)',
 'vysokoškolské vzdelanie - 1. stupeň (Bc.)',
 'vysokoškolské vzdelanie - 2. stupeň (Ing.; Mgr.; MUDr.; a i.)',
 'vysokoškolské vzdelanie - 3. stupeň (PhD.; a i.)',
 'vyššie odborné vzdelanie (bližšie neuvedené)',
 'vyššie odborné vzdelanie nadstavbové (maturita absolventov učebných odborov stredných odborných škôl)',
 'vyššie odborné vzdelanie pomaturitné (pomaturitné kvalifikačné)',
 'vyššie odborné vzdelanie vyššie odborné (absolventská skúška, absolventský diplom)',
 'základné vzdel

set()

### Count functions
- `count_number_age_based` - number
- `count_category_percent_age_based` - total is category
- `count_filtered_percent_age_based` - filtered / not filtered
- `count_age_percent_age_based` - total is all at the same age

In [38]:
def count_number_age_based(data, query, feature=None):
    """
    Counts the number of people in `data`
    filtered by `query` grouped by `feature`

    Parameters:
        data (DataFrame): The input DataFrame containing the data to count.
        query (str): The query string to filter the data.
        feature (str, optional): The column name to group by,
            determines categories (optional).
    """
    filtered = data.query(query)
    if feature is None:
        aggregated = filtered.groupby(['Vek'])['abs.'].sum().rename('number').reset_index()
    else:
        aggregated = filtered.groupby(['Vek', feature])['abs.'].sum().rename('number').reset_index()
    return aggregated

def count_category_percent_age_based(data, query, feature=None):
    """
    Counts percent of people in `data` filtered
    by `query` grouped by `feature`
    Percent is counted as number of people
    by age divided by the number of all people
    in the same category

    Parameters:
        data (DataFrame): The input DataFrame containing the data to count.
        query (str): The query string to filter the data.
        feature (str, optional): The column name to group by,
            determines categories (optional).
    """
    filtered = data.query(query)
    if feature is None:
        aggregated = filtered.groupby(['Vek'])['abs.'].sum().rename('number').reset_index()
        total = filtered['abs.'].sum()
        aggregated['percent'] = aggregated['number'] / total * 100
    else:
        aggregated = filtered.groupby(['Vek', feature])['abs.'].sum().rename('number').reset_index()
        total = filtered.groupby([feature])['abs.'].sum().rename('total')
        aggregated = pd.merge(aggregated, total, on=feature)
        aggregated['percent'] = aggregated['number'] / aggregated['total'] * 100
    # aggregated = aggregated[aggregated['count'] > 10]
    return aggregated

def count_filtered_percent_age_based(data, query, feature=None):
    """
    Counts percent of people in `data` filtered
    by `query` grouped by `feature`
    Percent is counted as number of filtered
    people divided by not filtered people
    in the same category and age

    Parameters:
        data (DataFrame): The input DataFrame containing the data to count.
        query (str): The query string to filter the data.
        feature (str, optional): The column name to group by,
            determines categories (optional).
    """
    filtered = data.query(query)
    if feature is None:
        aggregated = filtered.groupby(['Vek'])['abs.'].sum().rename('number').reset_index()
        total = data.groupby(['Vek'])['abs.'].sum()
        aggregated['percent'] = aggregated['number'] / total * 100
    else:
        aggregated = filtered.groupby(['Vek', feature])['abs.'].sum().rename('number')
        total = data.groupby(['Vek', feature])['abs.'].sum().loc[aggregated.index]
        percent = aggregated / total * 100
        aggregated = aggregated.reset_index()
        percent = percent.reset_index(name='percent')
        aggregated['percent'] = percent.reset_index()['percent']
    # aggregated = aggregated[aggregated['abs.'] > 10]
    return aggregated

def count_age_percent_age_based(data, query, feature=None):
    """
    Counts percent of people in `data` filtered
    by `query` grouped by `feature`
    Percent is counted as number of people divided
    by number of people at the same age

    Parameters:
        data (DataFrame): The input DataFrame containing the data to count.
        query (str): The query string to filter the data.
        feature (str, optional): The column name to group by,
            determines categories (optional).
    """
    filtered = data.query(query)
    if feature is None:
        aggregated = filtered.groupby(['Vek'])['abs.'].sum().rename('number').reset_index()
        total = data.groupby(['Vek'])['abs.'].sum()
        aggregated['percent'] = aggregated['number'] / total * 100
    else:
        aggregated = filtered.groupby(['Vek', feature])['abs.'].sum().rename('number').reset_index()
        total = filtered.groupby('Vek')['abs.'].sum().rename('total').reset_index()
        aggregated = pd.merge(aggregated, total, on='Vek')
        aggregated['percent'] = aggregated['number'] / aggregated['total'] * 100
    return aggregated

data = count_age_percent_age_based(table_40, 'Vzdelanie == "vysokoškolské vzdelanie - 1. stupeň (Bc.)"', feature='Názov kraja')
px.line(data, x='Vek', y='percent', color='Názov kraja', hover_data=['number'])

## Interactive `dash` plot by age

Data is filtered and than optionally grouped by feature  
Count function can be choosed from
- Number
- Category percent - total is category
- Filtered percent - filtered / not filtered
- Age percent - total is all at the same age

Filtering can be done by and features can be
- Vzdelanie
- Grouped edu
- Názov kraja
- Pohlavie
- Súčasná ekonomická aktivita

In [71]:
properties = [
    'Súčasná ekonomická aktivita',
    'Vzdelanie',
    # 'Názov oblasti',
    'Názov kraja',
    'Pohlavie',
    grouped_edu,
]
unique_data = [table_40[property].unique() for property in properties]
# selected_options = [list(range(len(unique_data[i]))) for i in range(len(properties))]
selected_options = [[0] for i in range(len(properties))]

### Configuring
  
As filling data each time from scratch by hand can be frustrating it is possible to import and export filtering options. Only filtering options are ready to be im/exported. **Remember** to change the file name in the next cell.

`%%script true` at the top of the code block means I do not want to accidentally run it. To run the cell just comment this line. **Do not forget** to uncomment it after.

As for now `properties` structure has changed, so old configs won't work, but with new everything ones will work

In [40]:
# selected_options_source = 'vyssie_vzdelanie.txt'
selected_options_source = 'vyssie_vzdelanie.txt'

In [41]:
%%script true
# Export
print(*[' '.join(map(str, s)) for s in selected_options], sep='\n', file=open(selected_options_source, 'w'))

In [42]:
%%script true
# Import
with open(selected_options_source, 'r') as file:
    for i, line in enumerate(file.readlines()):
        selected_options[i] = list(map(int, line.split()))
display(selected_options)

In [43]:
%%script true
# quickly setting up options
selected_options[0] = list(range(0))
selected_options[3] = []
selected_options[1] = list(range(8))
selected_options[2] = [0, 1]

In [72]:
# %%script true # Skip
app = Dash(__name__)

checklists = [
    html.Div([
        html.H4(f"Select {properties[i]}"),
        dcc.Checklist(
            id=f'selected_{i}',
            options=[{'label': v, 'value': i} for i, v in enumerate(unique_data[i])],
            value=selected_options[i]
        )
    ])
    for i in range(len(properties))
]

app.layout = html.Div(
    style=style,
    children=[
        html.Div(checklists[::2], style={'width': '50%', 'display': 'inline-block'}),
        html.Div(checklists[1::2], style={'width': '50%', 'display': 'inline-block'}),
        html.Br(),
        html.Br(),
        dcc.Graph(id='line-plot'),
        html.Div([
            html.Div([
                html.H4('Enter title'),
                dcc.Input(id='title', ),
                dcc.Checklist(
                    id='checkbox',
                    options=[
                        { 'label': 'Add markers', 'value': True }
                    ],
                    value=[]
                )
            ], style={'flex': 1, 'padding': 10}),
            html.Div([
                html.H4('Select feature'),
                dcc.RadioItems(
                    id='feature',
                    options=[{ 'label': 'No', 'value': None }] + [{ 'label': v, 'value': i } for i, v, in enumerate(properties)],
                    value=None,
                ),
            ], style={'flex': 1, 'padding': 10}),
            html.Div([
                html.H4('Select count method'),
                dcc.RadioItems([
                    { 'value': i, 'label': l }
                    for i, l in enumerate(['Number', 'Category percent', 'Filtered percent', 'Age percent'])
                ], 0, id='count-method'),
            ], style={'flex': 1, 'padding': 10}),
        ], style={'display': 'flex', 'flexDirection': 'row'})
    ]
)

@app.callback(
    Output('line-plot', 'figure'),
    [
        Input('feature', 'value'),
        Input('title', 'value'),
        Input('count-method', 'value'),
        Input('checkbox', 'value'),
    ]
     + [Input(f'selected_{i}', 'value') for i in range(len(properties))]
)
def update_figure(feature, title, count_method, checkbox, *arg):
    selected_options[:] = list(arg)
    query = ' and '.join([
        '(' + ' or '.join([f"`{properties[i]}` == '{unique_data[i][s]}'" for s in selected]) + ')'
        for i, selected in enumerate(arg)
        if len(selected) > 0
    ])
    count_functions = [count_number_age_based, count_category_percent_age_based, 
                    count_filtered_percent_age_based, count_age_percent_age_based]
    count_function = count_functions[count_method]
    feature = properties[feature] if feature is not None else None
    data = count_function(table_40, query, feature=feature)
    if count_method == 0:
        figure = px.line(data, x='Vek', y='number', color=feature)
        figure.update_layout(xaxis_title='Age', yaxis_title='Number of people', title=title)
    else:
        figure = px.line(data, x='Vek', y='percent', color=feature, hover_data=['number'])
        figure.update_layout(xaxis_title='Age', yaxis_title='Percent of people', title=title)
    figure.update_traces(mode='lines' + ('+markers' if len(checkbox) > 0 else ''))

    return figure

app.run_server(mode='inline', port=8053)